# Wildfire Risk Visualization with Kepler.gl

This notebook visualizes wildfire data using Kepler.gl with automatic styling.

## Setup

In [1]:
# Suppress keplergl pkg_resources deprecation warning
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='keplergl')
warnings.filterwarnings('ignore', category=UserWarning, module='jupyter_client')

import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
from pathlib import Path

## Load Data

In [2]:
# Load GeoJSON files from ETL output
data_dir = Path('../data/processed')
fires_gdf = gpd.read_file(data_dir / 'active_fires.geojson')
buffers_gdf = gpd.read_file(data_dir / 'fire_buffers.geojson')

# Fill NaN values to avoid JSON serialization issues
# Use numeric_only to avoid object dtype downcasting warning
fires_gdf = fires_gdf.fillna({col: 0 for col in fires_gdf.select_dtypes(include=['number']).columns})
buffers_gdf = buffers_gdf.fillna({col: 0 for col in buffers_gdf.select_dtypes(include=['number']).columns})

print(f"Loaded {len(fires_gdf)} fire detections")
print(f"Loaded {len(buffers_gdf)} buffer zones")

Loaded 2 fire detections
Loaded 3 buffer zones


## Visualize with Kepler.gl

In [3]:
# Create map with automatic center/zoom based on data bounds
# Calculate center from fire locations
center_lat = fires_gdf.geometry.y.mean()
center_lon = fires_gdf.geometry.x.mean()

# Create config with proper centering
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': center_lat,
            'longitude': center_lon,
            'zoom': 8  # Good zoom level for fire viewing
        }
    }
}

map_viz = KeplerGl(height=600, config=config)
map_viz.add_data(data=fires_gdf, name='fires')
map_viz.add_data(data=buffers_gdf, name='buffers')

print(f'Map centered at: ({center_lat:.4f}, {center_lon:.4f})')
print(f'Zoom level: 8')
map_viz

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'fires': {'index': [0, 1], 'columns': ['latitude', 'longitude', 'bright_ti4', 'scan', 'track', …

In [4]:
# Export to HTML for sharing
output_path = Path('../wildfire_map.html')
map_viz.save_to_html(file_name=str(output_path), read_only=False)
print(f"✓ Map exported to {output_path.absolute()}")

Map saved to ../wildfire_map.html!
✓ Map exported to /Users/bryan/dev/this_is_fine/notebooks/../wildfire_map.html


## Programmatic Configuration (Optional)

If you need to programmatically configure the map instead of using auto-styling, you can build a config dictionary:

In [5]:
# Example: Programmatically set point radius and colors
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 37.5,
            'longitude': -95.0,
            'zoom': 4
        },
        'visState': {
            'layers': [
                {
                    'id': 'fires_layer',
                    'type': 'point',
                    'config': {
                        'dataId': 'fires',
                        'label': 'Active Fires',
                        'color': [255, 100, 100],  # Red
                        'columns': {
                            'lat': 'latitude',
                            'lng': 'longitude'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'radius': 10,  # Point radius in pixels
                            'opacity': 0.8,
                            'outline': False,
                            'thickness': 2,
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'radiusRange': [5, 20],  # Min/max radius when sizing by field
                        }
                    },
                    'visualChannels': {
                        'colorField': {
                            'name': 'risk_score',
                            'type': 'real'
                        },
                        'colorScale': 'quantile',
                        'sizeField': {
                            'name': 'frp',
                            'type': 'real'
                        },
                        'sizeScale': 'sqrt'
                    }
                },
                {
                    'id': 'buffers_layer',
                    'type': 'geojson',
                    'config': {
                        'dataId': 'buffers',
                        'label': 'Risk Buffers',
                        'color': [100, 100, 255],  # Blue
                        'columns': {
                            'geojson': 'geometry'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.3,
                            'strokeOpacity': 0.8,
                            'thickness': 1,
                            'strokeColor': [50, 50, 200],
                            'colorRange': {
                                'name': 'Ice And Fire',
                                'type': 'diverging',
                                'category': 'Uber',
                                'colors': ['#7F1941', '#D50255', '#FEAD54', '#FEEDB1', '#B1E4E3', '#0E7C7B']
                            },
                            'strokeColorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'radius': 10,
                            'sizeRange': [0, 10],
                            'radiusRange': [0, 50],
                            'heightRange': [0, 500],
                            'elevationScale': 5,
                            'stroked': True,
                            'filled': True,
                            'enable3d': False,
                            'wireframe': False
                        }
                    },
                    'visualChannels': {
                        'colorField': {
                            'name': 'risk_category',
                            'type': 'string'
                        },
                        'colorScale': 'ordinal',
                        'strokeColorField': None,
                        'strokeColorScale': 'quantile',
                        'sizeField': None,
                        'sizeScale': 'linear',
                        'heightField': None,
                        'heightScale': 'linear',
                        'radiusField': None,
                        'radiusScale': 'linear'
                    }
                }
            ]
        }
    }
}

# Create map with programmatic config
map_configured = KeplerGl(
    height=600,
    data={'fires': fires_gdf, 'buffers': buffers_gdf},
    config=config
)
map_configured

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 37.5, 'longitude': -95.0, 'zoom': 4}, 'v…

### Key Configuration Options

**Point Layer (`type: 'point'`):**
- `radius`: Fixed point size in pixels
- `radiusRange`: [min, max] when sizing by a field
- `opacity`: Layer transparency (0-1)
- `colorField`: Field to color by (e.g., 'risk_score')
- `sizeField`: Field to size by (e.g., 'frp')
- `colorScale`: 'quantile', 'quantize', 'ordinal', 'linear'

**GeoJSON Layer (`type: 'geojson'`):**
- `opacity`: Fill transparency
- `strokeOpacity`: Outline transparency
- `thickness`: Outline width
- `filled`: Show polygon fill
- `stroked`: Show polygon outline
- `colorField`: Field to color by (e.g., 'risk_category')

**Color Ranges:**
- Sequential: 'Global Warming', 'Uber Pool', 'Teal', 'Purple'
- Diverging: 'Ice And Fire', 'Outback', 'Pink Wine'
- Custom: Provide array of hex colors

## Export to HTML

Save as standalone HTML file for sharing:

In [6]:
# Export the auto-styled map
output_path = Path('../wildfire_map.html')
map_viz.save_to_html(file_name=str(output_path), read_only=False)
print(f"Map exported to {output_path.absolute()}")

Map saved to ../wildfire_map.html!
Map exported to /Users/bryan/dev/this_is_fine/notebooks/../wildfire_map.html


## Air Quality Visualization

If you've enriched your fire data with `--aqi` or `--purpleair` flags, you can create visualizations showing air quality impacts.

### Available Air Quality Columns:
- **AirNow (EPA)**: `aqi`, `aqi_category`, `aqi_parameter`
- **PurpleAir**: `pa_pm25`, `pa_pm25_60min`, `pa_sensor_count`

In [7]:
# Check which air quality data is available
print('Air Quality Data Summary')
print('=' * 60)

# Check AirNow AQI
if 'aqi' in fires_gdf.columns:
    aqi_available = fires_gdf['aqi'].notna().sum()
    print(f'✓ AirNow AQI data: {aqi_available}/{len(fires_gdf)} fires')
    if aqi_available > 0:
        # Convert to numeric, handling any string values
        aqi_numeric = pd.to_numeric(fires_gdf['aqi'], errors='coerce')
        print(f'  AQI range: {aqi_numeric.min():.0f} - {aqi_numeric.max():.0f}')
        print(f'  Categories: {fires_gdf["aqi_category"].value_counts().to_dict()}')
else:
    print('  No AirNow data (run with --aqi flag)')

# Check PurpleAir
print()
if 'pa_pm25' in fires_gdf.columns:
    pa_available = fires_gdf['pa_pm25'].notna().sum()
    print(f'✓ PurpleAir PM2.5 data: {pa_available}/{len(fires_gdf)} fires')
    if pa_available > 0:
        # Convert to numeric, handling any string values
        pm25_numeric = pd.to_numeric(fires_gdf['pa_pm25'], errors='coerce')
        print(f'  PM2.5 range: {pm25_numeric.min():.1f} - {pm25_numeric.max():.1f} μg/m³')
        avg_sensors = pd.to_numeric(fires_gdf['pa_sensor_count'], errors='coerce').mean()
        print(f'  Avg sensors per fire: {avg_sensors:.1f}')
else:
    print('  No PurpleAir data (run with --purpleair flag)')

Air Quality Data Summary
✓ AirNow AQI data: 2/2 fires
  AQI range: 21 - 21
  Categories: {'Good': 2}

✓ PurpleAir PM2.5 data: 1/2 fires
  PM2.5 range: 2.4 - 2.4 μg/m³
  Avg sensors per fire: 0.5


### Visualization Option 1: Color Fires by AQI

Color fire points by their Air Quality Index to show which fires are causing the worst air quality.

In [8]:
# Create map colored by AQI
if 'aqi' in fires_gdf.columns and fires_gdf['aqi'].notna().any():
    # Calculate center from fire locations
    center_lat = fires_gdf.geometry.y.mean()
    center_lon = fires_gdf.geometry.x.mean()
    
    config = {
        'version': 'v1',
        'config': {
            'mapState': {
                'latitude': center_lat,
                'longitude': center_lon,
                'zoom': 8
            }
        }
    }
    
    map_aqi = KeplerGl(height=600, config=config)
    map_aqi.add_data(data=fires_gdf, name='fires_aqi')
    print('💨 Map created! Click on points to see AQI values')
    print(f'   Centered at: ({center_lat:.4f}, {center_lon:.4f})')
    print('   Suggested: Color by "aqi" field, size by "frp" field')
    map_aqi
else:
    print('⚠️  No AQI data available. Run ETL with --aqi flag first:')
    print('   uv run wildfire fetch --region colorado --aqi')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
💨 Map created! Click on points to see AQI values
   Suggested: Color by "aqi" field, size by "frp" field


### Visualization Option 2: PurpleAir PM2.5 Heatmap

Create a heatmap showing PM2.5 concentrations from PurpleAir sensors near fires.

In [9]:
# Create heatmap of PM2.5 values
if 'pa_pm25' in fires_gdf.columns and fires_gdf['pa_pm25'].notna().any():
    # Filter to only fires with PurpleAir data
    fires_with_pa = fires_gdf[fires_gdf['pa_pm25'].notna()].copy()
    
    # Calculate center
    center_lat = fires_with_pa.geometry.y.mean()
    center_lon = fires_with_pa.geometry.x.mean()
    
    config = {
        'version': 'v1',
        'config': {
            'mapState': {
                'latitude': center_lat,
                'longitude': center_lon,
                'zoom': 8
            }
        }
    }
    
    map_pm25 = KeplerGl(height=600, config=config)
    map_pm25.add_data(data=fires_with_pa, name='pm25_data')
    
    print(f'💨 Map created with {len(fires_with_pa)} points with PM2.5 data')
    print(f'   Centered at: ({center_lat:.4f}, {center_lon:.4f})')
    print()
    print('   📊 To create a HEATMAP:')
    print('   1. Click on "pm25_data" layer in the sidebar')
    print('   2. Change layer type from "Point" to "Heatmap"')
    print('   3. Set Weight by: "pa_pm25"')
    print('   4. Adjust Radius and Intensity sliders as needed')
    print()
    print('   💡 Or use Point layer:')
    print('   - Color by: "pa_pm25" (PM2.5 concentration)')
    print('   - Size by: "pa_sensor_count" (data quality)')
    print()
    print('   PM2.5 Reference (μg/m³):')
    print('   - 0-50: Good')
    print('   - 51-100: Moderate')
    print('   - 101-150: Unhealthy for Sensitive Groups')
    print('   - 151+: Unhealthy')
    map_pm25
else:
    print('⚠️  No PurpleAir data available. Run ETL with --purpleair flag first:')
    print('   uv run wildfire fetch --region colorado --purpleair')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
💨 Heatmap created with 1 points
   Suggested layer settings:
   - Layer type: Heatmap or Point
   - Color by: "pa_pm25" (PM2.5 concentration)
   - Size by: "pa_sensor_count" (data quality indicator)

   PM2.5 Reference:
   - 0-50: Good
   - 51-100: Moderate
   - 101-150: Unhealthy for Sensitive Groups
   - 151+: Unhealthy


### Visualization Option 3: Combined Multi-Layer Map

Create a comprehensive map with fire risk, AQI, and PM2.5 data all in one view.

In [10]:
# Create comprehensive air quality + fire map
# Calculate center from fire locations
center_lat = fires_gdf.geometry.y.mean()
center_lon = fires_gdf.geometry.x.mean()

config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': center_lat,
            'longitude': center_lon,
            'zoom': 8
        }
    }
}

map_combined = KeplerGl(height=600, config=config)

# Add all data layers
map_combined.add_data(data=fires_gdf, name='fires')
map_combined.add_data(data=buffers_gdf, name='risk_buffers')

print(f'Map centered at: ({center_lat:.4f}, {center_lon:.4f})')
print()

# Add AQI layer if available
if 'aqi' in fires_gdf.columns and fires_gdf['aqi'].notna().any():
    fires_aqi = fires_gdf[fires_gdf['aqi'].notna()].copy()
    map_combined.add_data(data=fires_aqi, name='aqi_points')
    print(f'✓ Added {len(fires_aqi)} fires with AirNow AQI data')

# Add PurpleAir layer if available
if 'pa_pm25' in fires_gdf.columns and fires_gdf['pa_pm25'].notna().any():
    fires_pa = fires_gdf[fires_gdf['pa_pm25'].notna()].copy()
    map_combined.add_data(data=fires_pa, name='pm25_heatmap')
    print(f'✓ Added {len(fires_pa)} fires with PurpleAir PM2.5 data')

print()
print('💡 Suggested layer configuration:')
print('   1. Risk Buffers (polygons) - colored by risk_category')
print('   2. Fires (points) - colored by risk_score')
print('   3. AQI Points (points) - colored by aqi, sized by frp')
print('   4. PM2.5 Heatmap (change to heatmap type) - weight by pa_pm25')
print()
print('Toggle layers on/off in the sidebar to explore different views!')

map_combined

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
✓ Added 2 fires with AirNow AQI data
✓ Added 1 fires with PurpleAir PM2.5 data

💡 Suggested layer configuration:
   1. Risk Buffers (polygons) - colored by risk_category
   2. Fires (points) - colored by risk_score
   3. AQI Points (points) - colored by aqi, sized by frp
   4. PM2.5 Heatmap (heatmap) - weight by pa_pm25

Toggle layers on/off in the sidebar to explore different views!


KeplerGl(data={'fires': {'index': [0, 1], 'columns': ['latitude', 'longitude', 'bright_ti4', 'scan', 'track', …

### Export Air Quality Maps

Save any of the above maps as standalone HTML files:

In [11]:
# Export the combined air quality map
output_path = Path('../wildfire_airquality_map.html')
map_combined.save_to_html(file_name=str(output_path), read_only=False)
print(f'✓ Air quality map exported to {output_path.absolute()}')
print('\nOpen this file in a web browser to share with others!')

Map saved to ../wildfire_airquality_map.html!
✓ Air quality map exported to /Users/bryan/dev/this_is_fine/notebooks/../wildfire_airquality_map.html

Open this file in a web browser to share with others!
